# Tutorial Notebook on MLFlow for evaluating ML and LLM models

In [8]:
%load_ext autoreload
%autoreload 2

import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
env_loader = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [9]:
import pandas as pd
import mlflow

## Basic Question-Answering Evaluation

In [12]:
eval_df = pd.DataFrame(
    {
        "inputs": [
            "How does useEffect() work?",
            "What does the static keyword in a function mean?",
            # "What does the 'finally' block in Python do?",
            # "What is the difference between multiprocessing and multithreading?",
        ],
        "ground_truth": [
            "The useEffect() hook tells React that your component needs to do something after render. React will remember the function you passed (we’ll refer to it as our “effect”), and call it later after performing the DOM updates.",
            "Static members belongs to the class, rather than a specific instance. This means that only one instance of a static member exists, even if you create multiple objects of the class, or if you don't create any. It will be shared by all objects.",
            # "'Finally' defines a block of code to run when the try... except...else block is final. The finally block will be executed no matter if the try block raises an error or not.",
            # "Multithreading refers to the ability of a processor to execute multiple threads concurrently, where each thread runs a process. Whereas multiprocessing refers to the ability of a system to run multiple processors in parallel, where each processor can run one or more threads.",
        ],
    }
)

In [19]:
with mlflow.start_run() as run:
    system_prompt = "Answer the following question in two sentences"
    basic_qa_model = mlflow.openai.log_model(
        model="gpt-4o-mini",
        task=openai.chat.completions,
        artifact_path="model",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": "{question}"},
        ],
    )
    results = mlflow.evaluate(
        basic_qa_model.model_uri,
        eval_df,
        targets="ground_truth",  # specify which column corresponds to the expected output
        model_type="question-answering",  # model type indicates which metrics are relevant for this task
        evaluators="default",
    )
mlflow.end_run()
results.metrics


2024/10/29 12:28:55 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/10/29 12:28:57 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/10/29 12:28:57 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: ModuleNotFoundError("No module named 'evaluate'")), skipping metric logging.
2024/10/29 12:28:57 WARNING mlflow.models.evaluation.default_evaluator: Did not log builtin metric 'toxicity' because it returned None.
2024/10/29 12:28:57 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: ModuleNotFoundError("No module named 'evaluate'")), skipping metric logging.
2024/10/29 12:28:57 WARNING mlflow.models.evaluation.default_evaluator: Did not log builtin metric 'toxicity' because it returned None.


{'flesch_kincaid_grade_level/v1/mean': 15.700000000000001,
 'flesch_kincaid_grade_level/v1/variance': 0.8100000000000006,
 'flesch_kincaid_grade_level/v1/p90': 16.42,
 'ari_grade_level/v1/mean': 18.95,
 'ari_grade_level/v1/variance': 0.20249999999999935,
 'ari_grade_level/v1/p90': 19.31,
 'exact_match/v1': 0.0}

In [22]:
results.tables["eval_results_table"]

,inputs,ground_truth,outputs,token_count,flesch_kincaid_grade_level/v1/score,ari_grade_level/v1/score,answer_similarity/v1/score,answer_similarity/v1/justification
0,How does useEffect() work?,The useEffect() hook tells React that your com...,The `useEffect()` hook in React allows you to ...,64,15.8,18.2,5,The output provided by the model aligns closel...
1,What does the static keyword in a function mean?,"Static members belongs to the class, rather th...",The `static` keyword in a function means that ...,58,13.0,15.8,3,The output provides a correct explanation of t...


## LLM-judged correctness with OpenAI GPT-4

In [20]:
from mlflow.metrics.genai import EvaluationExample, answer_similarity

# Create an example to describe what answer_similarity means like for this problem.
example = EvaluationExample(
    input="What is MLflow?",
    output="MLflow is an open-source platform for managing machine "
    "learning workflows, including experiment tracking, model packaging, "
    "versioning, and deployment, simplifying the ML lifecycle.",
    score=4,
    justification="The definition effectively explains what MLflow is "
    "its purpose, and its developer. It could be more concise for a 5-score.",
    grading_context={
        "targets": "MLflow is an open-source platform for managing "
        "the end-to-end machine learning (ML) lifecycle. It was developed by Databricks, "
        "a company that specializes in big data and machine learning solutions. MLflow is "
        "designed to address the challenges that data scientists and machine learning "
        "engineers face when developing, training, and deploying machine learning models."
    },
)

# Construct the metric using OpenAI GPT-4 as the judge
answer_similarity_metric = answer_similarity(model="openai:/gpt-4", examples=[example])

print(answer_similarity_metric)

EvaluationMetric(name=answer_similarity, greater_is_better=True, long_name=answer_similarity, version=v1, metric_details=
Task:
You must return the following fields in your response in two lines, one below the other:
score: Your numerical score for the model's answer_similarity based on the rubric
justification: Your reasoning about the model's answer_similarity score

You are an impartial judge. You will be given an input that was sent to a machine
learning model, and you will be given an output that the model produced. You
may also be given additional information that was used by the model to generate the output.

Your task is to determine a numerical score called answer_similarity based on the input and output.
A definition of answer_similarity and a grading rubric are provided below.
You must use the grading rubric to determine your score. You must also justify your score.

Examples could be included below for reference. Make sure to use them as references and to
understand them be

In [21]:
with mlflow.start_run() as run:
    results = mlflow.evaluate(
        basic_qa_model.model_uri,
        eval_df,
        targets="ground_truth",
        model_type="question-answering",
        evaluators="default",
        extra_metrics=[answer_similarity_metric],  # use the answer similarity metric created above
    )
results.metrics

2024/10/29 12:32:01 INFO mlflow.models.evaluation.default_evaluator: Computing model predictions.
2024/10/29 12:32:03 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/10/29 12:32:03 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: ModuleNotFoundError("No module named 'evaluate'")), skipping metric logging.
2024/10/29 12:32:03 WARNING mlflow.models.evaluation.default_evaluator: Did not log builtin metric 'toxicity' because it returned None.


  0%|          | 0/1 [00:00<?, ?it/s]

2024/10/29 12:32:06 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: ModuleNotFoundError("No module named 'evaluate'")), skipping metric logging.
2024/10/29 12:32:06 WARNING mlflow.models.evaluation.default_evaluator: Did not log builtin metric 'toxicity' because it returned None.


  0%|          | 0/2 [00:00<?, ?it/s]

{'flesch_kincaid_grade_level/v1/mean': 14.4,
 'flesch_kincaid_grade_level/v1/variance': 1.960000000000001,
 'flesch_kincaid_grade_level/v1/p90': 15.520000000000001,
 'ari_grade_level/v1/mean': 17.0,
 'ari_grade_level/v1/variance': 1.4399999999999984,
 'ari_grade_level/v1/p90': 17.96,
 'exact_match/v1': 0.0,
 'answer_similarity/v1/mean': 4.0,
 'answer_similarity/v1/variance': 1.0,
 'answer_similarity/v1/p90': 4.8}

# Deepeval

In [12]:
import deepeval

from deepeval import assert_test
from deepeval.test_case import LLMTestCase
from deepeval.metrics import AnswerRelevancyMetric

def test_answer_relevancy():
    answer_relevancy_metric = AnswerRelevancyMetric(threshold=0.5)
    test_case = LLMTestCase(
        input="What if these shoes don't fit?",
        # Replace this with the actual output of your LLM application
        actual_output="", # "We offer a 20-day full refund at no extra cost.",
        retrieval_context=["All customers are eligible for a 30 day full refund at no extra cost."]
    )
    assert_test(test_case, [answer_relevancy_metric])

In [14]:
print(test_answer_relevancy())

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 1 test case(s) in parallel: |██████████|100% (1/1) [Time Taken: 00:42, 42.01s/test case]

None
